SEP 775 Final Project Code (Implementation of AI Teaching Assistant)

Submitted By Group-4 :

Rutvik, Damjibhai Roy – 400490159

Meetkumar Patel - 400547066

Mridu - 400547058

Srey Patel - 400546143

In [2]:
# Install neccessary libraries

!pip install langchain==0.1.14
!pip install sentence-transformers==2.6.1
!pip install faiss-cpu==1.8.0
!pip install pdfminer.six==20231228
!pip install llama_cpp_python==0.2.58
!pip install tiktoken==0.6.0
!pip install gdown==5.1.0

  Using cached langchain-0.1.14-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.31-py3-none-any.whl.metadata (8.4 kB)
Using cached langchain-0.1.14-py3-none-any.whl (812 kB)
Using cached langchain_community-0.0.31-py3-none-any.whl (1.9 MB)
   ---------------------------------------- 0.0/276.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/276.8 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/276.8 kB 660.6 kB/s eta 0:00:01
   ----------- --------------------------- 81.9/276.8 kB 762.6 kB/s eta 0:00:01
   -------------------- ------------------- 143.4/276.8 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------  276.5/276.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 276.8/276.8 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.33
    Uninstalling langchain-core-0.1.33:
      Successfully uninstalled langchain-core-0.1.33
  Att


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\rutvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
Using cached sentence_transformers-2.6.1-py3-none-any.whl (163 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\rutvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\rutvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\rutvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached llama_cpp_python-0.2.58-cp311-cp311-win_amd64.whl
  Attempting uninstall: llama_cpp_python
    Found existing installation: llama_cpp_python 0.2.57
    Uninstalling llama_cpp_python-0.2.57:
      Successfully uninstalled llama_cpp_python-0.2.57



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\rutvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\rutvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\rutvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# **1. Load Documents (PDF files of SEP 775 Course Material ) and LLM**

In [1]:
# Download llama-2 7B model from gdrive

import gdown

id='1fJPVfJssRO-PXaHKkxMe4BZ2PS4QwXBn'
url = f"https://drive.google.com/uc?id={id}"
output = "llama-2-7b-chat.Q4_K_M.gguf"
gdown.download(url, output)

Downloading...
From (original): https://drive.google.com/uc?id=1fJPVfJssRO-PXaHKkxMe4BZ2PS4QwXBn
From (redirected): https://drive.google.com/uc?id=1fJPVfJssRO-PXaHKkxMe4BZ2PS4QwXBn&confirm=t&uuid=333d2833-1dae-48c8-97da-c9da1e5bad27
To: c:\Users\rutvi\OneDrive\Desktop\NLP Project\Final_code\llama-2-7b-chat.Q4_K_M.gguf
100%|██████████| 4.08G/4.08G [10:47<00:00, 6.30MB/s]


'llama-2-7b-chat.Q4_K_M.gguf'

In [1]:
# Download all pdf files (SEP 775 Course Materials) from gdrive

import gdown

id="1PuOPV-TYxcfxmqwcUqenTSuPlXFVRd-P"
!mkdir '/All_lecture_pdfs'
output='/All_lecture_pdfs'
documents=gdown.download_folder(id=id,output=output, quiet=True)

The syntax of the command is incorrect.


# **2. Extract Text and Metadata**

In [2]:
import os
import time

# Function to get last modified time of pdf files
def get_modified_time(file):
  ti_m = os.path.getmtime(file)
  m_ti = time.ctime(ti_m)
  t_obj = time.strptime(m_ti)
  # Transforming the time object to a timestamp of ISO 8601 format
  T_stamp = time.strftime("%Y-%m-%d %H:%M:%S", t_obj)
  return T_stamp

def replace_newlines(text):
    # Replace newline and carriage return + line feed characters with spaces
    return text.replace('\n', ' ').replace('\r\n', ' ').replace('\x0c', ' ')

def fix_missing_spaces(text):
    # Split text into sentences
    sentences = text.split('. ')
    # Add space after period for each sentence
    fixed_text = '. '.join(sentence + (' ' if i < len(sentences) - 1 else '') for i, sentence in enumerate(sentences))
    return fixed_text

In [3]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

# Function to extract text from pdf files
def prepare_docs(pdf_docs):
    docs = []
    metadata = []
    content = []

    for pdf in pdf_docs:
      for page_number, page_layout in enumerate(extract_pages(pdf), start=1):
        # Extract text from the page layout
        text = ""
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                text += element.get_text()

        text = replace_newlines(text)
        text = fix_missing_spaces(text)
        doc_page = {'Title': pdf.split("\\")[-1] + " Page No: " + str(page_number),
                    'Last_modified_time': get_modified_time(pdf),
                    'Content': text,
                    'Source': "empty_url"}
        docs.append(doc_page)

    for doc in docs:
        content.append(doc["Content"])
        metadata.append({
            "Title": doc["Title"],
            "Last_modified_time": doc["Last_modified_time"],
            "Source": doc["Source"]
        })

    print("Content and metadata are extracted from the documents")

    return content, metadata

In [4]:
# Split extracted text into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=512,chunk_overlap=15)
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(split_docs)} passages")

    return split_docs

# **3. Vector Database**

In [5]:
# Create Vector database

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

# Embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

def ingest_into_vectordb(split_docs):
    embeddings = embedding_model
    db = FAISS.from_documents(split_docs, embeddings)
    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    print("Vector database is created")
    return db

C:\Users\rutvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\rutvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
content, metadata = prepare_docs(documents)

Content and metadata are extracted from the documents


In [7]:
split_docs = get_text_chunks(content, metadata)

Documents are split into 550 passages


In [8]:
vectordb=ingest_into_vectordb(split_docs)

Vector database is created


# **4. Database Retriever**

In [9]:
# Set retriver

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [10]:

# Test a user query

user_query="What is RNN?"
query_embedding = embedding_model.embed_query(user_query)
print(query_embedding)

[-0.12031889706850052, -0.07917103916406631, -0.045413147658109665, 0.03378615528345108, 0.008108881302177906, 0.056303657591342926, -0.010072475299239159, 0.022870969027280807, 0.035111624747514725, -0.06259121000766754, -0.11304843425750732, 0.10516740381717682, -0.031245367601513863, 0.05396154522895813, -0.12448606640100479, -0.03206890821456909, -0.0533745251595974, 0.020140431821346283, 0.006022085901349783, 0.023872800171375275, -0.03664499521255493, 0.016491316258907318, 0.027449458837509155, 0.02671530842781067, 0.1028013750910759, -0.05205776169896126, 0.03360937535762787, 0.003733858233317733, 0.0590813122689724, 0.017603503540158272, 0.10120708495378494, 0.04195006191730499, -0.015093994326889515, 0.059379693120718, -0.05930508300662041, 0.038471199572086334, -0.06145156919956207, -0.007195512298494577, -0.0004484507080633193, 0.010540989227592945, 0.01921147108078003, -0.016691172495484352, -0.0007071466534398496, -0.04399330914020538, 0.12619024515151978, 0.01370256952941

In [11]:
# Check retriver

docs = retriever.get_relevant_documents(user_query)

for i,doc in enumerate(docs):
  print(f"Retrived Chunk - {i+1} :",docs[i].page_content)
  print(" ")
  print("Title:",doc.metadata["Title"])
  print("Last_modified_time:",doc.metadata["Last_modified_time"])
  print("Source:",doc.metadata["Source"])
  print("-"*40)
  print(" ")

Retrived Chunk - 1 : Other RNN Architectures
 
Title: lecture03-RNNs and LLMs.pdf Page No: 71
Last_modified_time: 2024-04-02 08:45:55
Source: empty_url
----------------------------------------
 
Retrived Chunk - 2 : Terminology The RNN described in this lecture = simple/vanilla/Elman RNN You learned about other RNN flavors like LSTM and GRU and multi-layer RNNs By the end of the course: You will understand phrases like “stacked bidirectional LSTMs with residual connections and self-attention” 83
 
Title: lecture03-RNNs and LLMs.pdf Page No: 83
Last_modified_time: 2024-04-02 08:45:55
Source: empty_url
----------------------------------------
 
Retrived Chunk - 3 : Other RNN uses: RNNs can be used for sequence tagging e.g., part-of-speech tagging, named entity recognition DT JJ NN VBN IN DT NN the startled cat knocked over the vase 72
 
Title: lecture03-RNNs and LLMs.pdf Page No: 72
Last_modified_time: 2024-04-02 08:45:55
Source: empty_url
----------------------------------------
 


In [14]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **5. RAG Conversational Chain**

In [14]:
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# callback manager for word to word streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Load Llama-2 Model from directory

llama_llm = LlamaCpp(
model_path="llama-2-7b-chat.Q4_K_M.gguf",
n_gpu_layers=15000, 
n_threads=6,     # Number of CPU core
n_batch=512,
temperature=0.7,
f16_kv=True,
max_tokens=512,
top_p=0.95,
callback_manager=callback_manager,
n_ctx=4096,
verbose=True,
streaming=True)

5.1. Prompt Template and retrieved document's citation (Define a fuction to cite retrived chunks' sources with LLM's response)

In [27]:
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferWindowMemory

# Set up conversation memory to save chat history
memory=ConversationBufferWindowMemory(k=2,memory_key="chat_history", return_messages=True, input_key="question")

# Function to set up langchain chain for sequence of call
def get_conversational_chain():

    prompt_template ="""
    You are a helpful Teaching Assistant of the McMaster University and your name is 'Mac AI Assistant'.\n
    This is the conversation between a student and and 'Mac AI Assistant". your job is to answer the student's question.\n
    The question can be a new question or follow up. So, you must check the chat histroy given below before you answer the question.\n
    You must answer student's the question based on only context given below.\n
    If the question can not be answered using the information provided in the context, must answer with I don't know, don't try to make up an answer.\n
    Give answers in natural form, without giving context as of what you're doing internally.\n
    Use three sentences maximum. Keep the answer as concise as possible.\n
    If user question is more general for eaxmple 'Hi', 'Hi there!, 'Thanks', or 'How are you!', then asnwer them like a personal assistant of an user\n
    Always begin your answer with this dialog format:\n 'Mac AI Assistant: <your_Answer>' \n\n

    context:\n{context}\n

    Student's Question: \n{question}\n

    Chat history: \n{chat_history}\n

    Answer: """

    # Langchain Prompt template to configure prompt variable
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question","chat_history"])

    # Chain for sequence of call
    chain = load_qa_chain(llama_llm, chain_type="stuff", prompt=prompt, memory=memory)

    return chain


In [28]:
# Function for user input

def user_input(user_question):
  docs = retriever.get_relevant_documents(user_question)
  chain = get_conversational_chain()
  response = chain.invoke({"input_documents": docs, "question": user_question})
  return response

In [29]:
# Function to cite retrived resources    

def citation_function(response):
    print('\nCitation:')
    for source in response["input_documents"]:
        print(source.metadata['Title']," ","Source URL:",source.metadata['Source'])

In [30]:
# Create a loop to continuously interact with QA chain ( TO exit the loop - Enter exit)

while True:
    user_query_=input("User: ")
    print("User: ",user_query_)
    if user_query_.lower() in ['exit', 'quit']:
        print("Exiting the program.")
        break
    response=user_input(user_query_)
    citation_function(response)
    print("\n")

User:  Hi there!

Mac AI Assistant: Hello! How can I help you today?
Citation:
lecture09-Multimodal Models.pdf Page No: 79   Source URL: empty_url
lecture01-wordvecs1.pdf Page No: 43   Source URL: empty_url
lecture07-Prompting, Instruction Finetuning, and RLHF  (1).pdf Page No: 82   Source URL: empty_url


User:  what do you mean by RNN?

Mac AI Assistant: RNN stands for Recurrent Neural Network, a type of neural network that is particularly well-suited for processing sequential data, such as text or time series data. Other RNN flavors include LSTM, GRU, and multi-layer RNNs, which have additional features or architectural improvements that make them more powerful or efficient for certain tasks.
Citation:
lecture03-RNNs and LLMs.pdf Page No: 71   Source URL: empty_url
lecture03-RNNs and LLMs.pdf Page No: 83   Source URL: empty_url
lecture03-RNNs and LLMs.pdf Page No: 72   Source URL: empty_url


User:  EXIT
Exiting the program.


5.2. Include citation within the prompt template (Citation done By LLM according to instruction inside prompt)

In [23]:
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferWindowMemory

# Set up conversation memory
memory=ConversationBufferWindowMemory(k=2,memory_key="chat_history", return_messages=True, input_key="question")

def get_conversational_chain():

    prompt_template ="""
    You are a helpful Teaching Assistant of the McMaster University and your name is 'Mac AI Assistant'.\n
    This is the conversation between a student and and 'Mac AI Assistant". your job is to answer the student's question.\n
    The question can be a new question or follow up. So, you must check the chat histroy given below before you answer the question.\n
    You must answer student's the question based on only context given below.\n
    If the question can not be answered using the information provided in the context, must answer with I don't know, don't try to make up an answer.\n
    Give answers in natural form, without giving context as of what you're doing internally.\n
    Use three sentences maximum. Keep the answer as concise as possible.\n
    If an answer to the question is provided uisng the context data, it must be annotated with a citation at the end of the answer. should use the following format to cite all three sources specified in the Sources after context data. "\nCitation: \nsource-1.pdf Page No: xx-1  Source URL: xxx-1 \nsource-2.pdf Page No: xx-2  Source URL: xxx-2 \nsource-3.pdf Page No: xx-3  Source URL: xxx-3".
    If user question is more general for eaxmple 'Hi', 'Hi there!, 'Thanks', or 'How are you!', then asnwer them like a personal assistant of an user and do not need citation in the asnwer\n
    Always begin your answer with this dialog format:\n'Mac AI Assistant: <your_Answer>' \n\n

    ""context:\n{context}\n\n""

    ""Chat history: \n{chat_history}\n\n""

    ""Student's Question: \n{question}\n\n""

    Answer: """

    # Langchain Prompt template to configure prompt variable
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question","chat_history"])

    # chain for sequence of call
    chain = load_qa_chain(llama_llm, chain_type="stuff", prompt=prompt, memory=memory,verbose=False)

    return chain


In [24]:
from langchain.docstore.document import Document

# Function to add retrievd text and metadata to put into prompt
def Add_text_with_metadata(docs):
    text=""
    data=""

    for doc in docs:
        text+=doc.page_content+"\n"
        data+=doc.metadata['Title']+" Source URL: "+ doc.metadata['Source']+"\n"

    final_text=text +"Sources: \n"+data
    doc =  Document(page_content=f"{final_text}")

    return [doc]  

In [25]:
# Function for user input

def user_input(user_question):
  retrived_docs = retriever.get_relevant_documents(user_question)
  docs=Add_text_with_metadata(retrived_docs)
  chain = get_conversational_chain()
  response = chain.invoke({"input_documents": docs, "question": user_question})
  return response

In [26]:
# Create a loop to continuously interact with QA chain ( TO exit the loop - Enter exit)

while True:
    user_query_=input("User: ")
    print("User: ",user_query_)
    if user_query_.lower() in ['exit', 'quit']:
        print("Exiting the program.")
        break
    response=user_input(user_query_)
    print("\n")

User:  what do you mean by RNN?

Mac AI Assistant: Hi there! In the context of NLP, RNN stands for Recurrent Neural Network, a type of neural network that is particularly well-suited for processing sequential data, such as text or speech. RNNs use loops to feed information from previous time steps into the current step, allowing them to capture temporal dependencies in data. (Citation: lecture03-RNNs and LLMs.pdf Page No: 71)

User:  can you give course logistic in brief?
 Mac AI Assistant: Hi there! Course logistics in brief: Instructor: Hamidreza Mahyar • Head TA: Reza Namazi • Time: Wednesday 3:30pm–6:30pm • We put a lot of other important information on Avenue to Learn . Please read it! (Citation: lecture01-wordvecs1.pdf Page No: 5)

User:  what do you mean by Multimodels?

Mac AI Assistant: Hi there! In the context of NLP, multimodality refers to the combination of multiple modalities, such as text, images, speech, and others, in a single model or system. Multimodal models aim to 